#### 과제1) 환경공간정보서비스(https://egis.me.go.kr) 크롤링

In [4]:
import requests
from bs4 import BeautifulSoup 

In [ ]:
url='https://egis.me.go.kr/atlas/list.do'
response=requests.post(url)
response

In [ ]:
bs=BeautifulSoup(response.content, 'html.parser')

In [ ]:
sel=bs.select_one('div.gallery')

In [ ]:
datas=sel.select('div.gallery-item')

In [ ]:
datas

In [ ]:
mats=[]
for data in datas:
    title = data.select_one('.tit a').text.strip()
    clas = data.select_one('.cate').text.strip()
    content= data.select_one('.detail > dl:nth-child(1) > dd').text.strip()
    date=data.select_one('.right').text.strip()
    mats.append([title,clas,content,date])

### 과제2) 네이버 뉴스(정치) 가져와서 DB저장하기

In [ ]:
import requests
from bs4 import BeautifulSoup 

In [ ]:
url='https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid1=100&sid2=264'
response=requests.get(url)
response

In [ ]:
bs=BeautifulSoup(response.content, 'html.parser')

In [ ]:
conts=bs.select_one('.type06_headline').select('li')
conts

# html 살펴보기
<ul class="type06_headline">
<li>
    <dl>
        <dt class="photo">
            <a class="nclicks(pol.2ndcont)" href="https://n.news.naver.com/mnews/article/008/0004987260?sid=100">
            <img alt='尹대통령 "노동계·특정 정치세력, 계급갈등으로 사회 들여다봐"' height="72" onerror="javascript:this.src='https://ssl.pstatic.net/static.news/image/news/2009/noimage_106x72.png';" src="https://imgnews.pstatic.net/image/origin/008/2024/01/17/4987260.jpg?type=ofullfill106_72" width="106"/>
            </a>
        </dt>
        <dt>
            <a class="nclicks(pol.2ndcont)" href="https://n.news.naver.com/mnews/article/008/0004987260?sid=100">
								 尹대통령 "노동계·특정 정치세력, 계급갈등으로 사회 들여다봐"
								</a>
        </dt>
        <dd>
            <span class="lede">윤석열 대통령이 17일 "노동계나 특정 정치 세력이 자본가와 노동자라는 양극의 계급갈등으로 사회를 들여다보기 때문에 일관되고 합 …</span>
            <span class="writing">머니투데이</span>
            <span class="date is_new">8분전</span>
        </dd>
    </dl>
</li>

In [ ]:
news=[]
for new in conts:
    title=new.select_one('dt:nth-child(2) > a').text.strip()
    url= new.select_one('dt:nth-child(2) > a').get('href')
    pub=new.select_one('span.writing ').text.strip()
    news.append([title,pub,url])

#### 과제2) 완성-페이지 바꿔가며 크롤링 & DB저장까지

In [3]:
import pymysql

In [239]:
news = []
conn = pymysql.connect(user = 'maria',
                       password = '1234',
                       host = 'localhost',
                       port = 3306,
                       database = 'test_db')
cursor=conn.cursor()

In [241]:
for page in range(1,10):
    url=f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=264&sid1=100&date=20240117&page={page}'
    response=requests.get(url)
    if response.status_code == 200:
        bs=BeautifulSoup(response.content, 'html.parser')
        conts=bs.select_one('.type06_headline').select('li')
        for new in conts:
            try:
                title=new.select_one('dt:nth-child(2) > a').text.strip()
                url= new.select_one('dt:nth-child(2) > a').get('href')
                pub=new.select_one('span.writing ').text.strip()
                news.append([title,pub,url])
            except:
                continue
    else:
        print(response.status_code)

sql = '''insert into news_list values (%s, %s, %s)'''
for new in news: 
    try:
        cursor.execute(sql, new)
    except:
        print('errrrr')
        continue
conn.commit()

#### 과제2) 함수 만들기 (페이지 수가 매개변수)

In [22]:
def pol_news (n):
    news = []
    for page in range(1,n):
        url=f'https://news.naver.com/main/list.naver?mode=LS2D&mid=shm&sid2=264&sid1=100&date=20240117&page={page}'
        response=requests.get(url)
        if response.status_code == 200:
            bs=BeautifulSoup(response.content, 'html.parser')
            conts=bs.select_one('.type06_headline').select('li')
            for new in conts:
                try:
                    title=new.select_one('dt:nth-child(2) > a').text.strip()
                    url= new.select_one('dt:nth-child(2) > a').get('href')
                    pub=new.select_one('span.writing ').text.strip()
                    news.append([title,pub,url])
                    print(news, '\n')
                except:
                    continue
        else:
            print(response.status_code)


In [27]:
news_view=pol_news(2)

[['尹 "교권 확립돼야 학생 인권도 보장…해결방안 계속 찾을 것"(종합)', '이데일리', 'https://n.news.naver.com/mnews/article/018/0005656766?sid=100']] 

[['尹 "교권 확립돼야 학생 인권도 보장…해결방안 계속 찾을 것"(종합)', '이데일리', 'https://n.news.naver.com/mnews/article/018/0005656766?sid=100'], ['윤석열 대통령, 교육계 신년교례회 참석', '연합뉴스', 'https://n.news.naver.com/mnews/article/001/0014450845?sid=100']] 

[['尹 "교권 확립돼야 학생 인권도 보장…해결방안 계속 찾을 것"(종합)', '이데일리', 'https://n.news.naver.com/mnews/article/018/0005656766?sid=100'], ['윤석열 대통령, 교육계 신년교례회 참석', '연합뉴스', 'https://n.news.naver.com/mnews/article/001/0014450845?sid=100'], ['尹대통령 "교권 확립돼야 학생 인권도 보장돼"', '머니투데이', 'https://n.news.naver.com/mnews/article/008/0004987378?sid=100']] 

[['尹 "교권 확립돼야 학생 인권도 보장…해결방안 계속 찾을 것"(종합)', '이데일리', 'https://n.news.naver.com/mnews/article/018/0005656766?sid=100'], ['윤석열 대통령, 교육계 신년교례회 참석', '연합뉴스', 'https://n.news.naver.com/mnews/article/001/0014450845?sid=100'], ['尹대통령 "교권 확립돼야 학생 인권도 보장돼"', '머니투데이', 'https://n.news.naver.com/mnews/article/008/0004987378?sid=100'], ['"교원들 많이 만나